In [8]:
import pandas as pd

In [9]:

pima_diab = pd.read_csv(r"c:\\Users\\acreddy\\Desktop\\abc\\LogisticReg-PimaDiabetes\data\diabetes.csv")

# changing target column name from Outcome to Class
pima_diab.rename(columns={"Outcome": "Class"}, inplace=True)

In [10]:
pima_diab.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [11]:
pima_diab.shape

(768, 9)

In [13]:
pima_diab["Class"].value_counts()

0    500
1    268
Name: Class, dtype: int64

# Dealing with Imbalanced dataset:

In [207]:
# lets split class counts
# class_counts_0, class_counts_1= pima_diab["Class"].value_counts() 

In [208]:
# split the classes to apply the techniques
class_0= pima_diab[pima_diab["Class"]==0]
class_1= pima_diab[pima_diab["Class"]==1]

In [209]:
len(class_0)

500

In [210]:
len(class_1)

268

Note: under_sampling or over_sampling technqiues should apply after splitting the data in to train and test only.Its a best practice. 

# Under_Sampling

1. The 3 tecnhiques which i showed below for under_sampling does the same job ( removes the data from majority class) 

2. Under sampling is good when you have huge data.

# 1. under_sampling using simple python code 

In [139]:
# doing random under sampling the majority class removing some records:

# class_0_undersamp= class_0.sample(pima_diab["Class"].value_counts()[0])
class_0_undersamp= class_0.sample(n= len(class_1), replace= False) # replace=False, removes the duplicates

In [140]:
class_0_undersamp.shape

(268, 9)

In [141]:
class_1.shape

(268, 9)

In [145]:
# lets concatinate the both the dataframes
pima_diab_undersamp= pd.concat([class_0_undersamp, class_1])  # by default axis=0

In [146]:
pima_diab_undersamp.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
166,3,148,66,25,0,32.5,0.256,22,0
699,4,118,70,0,0,44.5,0.904,26,0


In [147]:
pima_diab_undersamp["Class"].value_counts()

0    268
1    268
Name: Class, dtype: int64

In [148]:
pima_diab_undersamp.shape

(536, 9)

#         2. Resampling(under_sampling) using sklearn

In [158]:
from sklearn.utils import resample

In [159]:
class_0_under= resample(class_0, replace= False, n_samples= len(class_1)) # replace=False, avoids duplicate datapoints.

In [160]:
class_0_under.shape

(268, 9)

In [161]:
pima_diab_sklearn_under= pd.concat([class_0_under, class_1])

In [162]:
pima_diab_sklearn_under.head(1)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
232,1,79,80,25,37,25.4,0.583,22,0


In [163]:
pima_diab_sklearn_under["Class"].value_counts()

0    268
1    268
Name: Class, dtype: int64

In [164]:
pima_diab_sklearn_under.shape

(536, 9)

# 3. under_sampling using Imblearn

In [171]:
from imblearn.under_sampling import RandomUnderSampler

In [173]:
rus= RandomUnderSampler(replacement=False, random_state=42)

In [28]:
# we cant fit imblearn on entire dataframe, we must split data in to X(predictors) and y(target)
X= pima_diab.drop("Class", axis=1)
y= pima_diab["Class"]

In [29]:
X.shape

(768, 8)

In [30]:
y.shape

(768,)

In [178]:
X_resampled, y_resampled= rus.fit_resample(X,y)

In [181]:
pima_diab_imblearn_under_df= pd.concat([X_resampled, y_resampled], axis=1)

In [182]:
pima_diab_imblearn_under_df.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
556,1,97,70,40,0,38.1,0.218,30,0
117,5,78,48,0,0,33.7,0.654,25,0


In [183]:
pima_diab_imblearn_under_df["Class"].value_counts()

0    268
1    268
Name: Class, dtype: int64

In [184]:
pima_diab_imblearn_under_df.shape

(536, 9)

# 4. under_sampling using inblearn- TomekLink

Note:  It identifies pairs of instances (Tomek Links) of different classes that are close to each other. Removes the 
       majority class instance from each Tomek Link. Focuses on the boundary between classes and removes instances 
        that may cause ambiguity in the decision boundary.
        This is designed for only under_sampling.

In [49]:
from imblearn.under_sampling import TomekLinks

In [52]:
tl = TomekLinks(sampling_strategy='majority')

In [55]:
X_tl, y_tl = tl.fit_resample(X, y)

In [59]:
pima_tl_under= pd.concat([X_tl, y_tl], axis=1)

In [60]:
pima_tl_under.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [61]:
pima_tl_under["Class"].value_counts()

0    445
1    268
Name: Class, dtype: int64

If you see its reduced very less data from majority class as its removes data closed with minority class(tomkelinks).  

# Over_Sampling

Note: unlike under_sampling, over_sampling increases the data of minority class. When you apply above 1st both techniques(except       imblearn) creates the duplicates of the data. But when you use imblearn(SMOTE) technique it creates the synthetic data. So       inblearn is the best techniques to avoid over fitting the model.

      Same those above 3 techniques we will see


# 1. over_sampling using simple python code

In [194]:
# doing random over sampling the minority class, So this method duplicates the existing data:

#class_1_oversamp= class_1.sample(pima_diab["Class"].value_counts()[0], replace=True)
class_1_oversamp= class_1.sample(n= len(class_0), replace=True) # replace=False, Oversampling cannot create new rows; it can only select existing rows.

In [195]:
class_1_oversamp["Class"].value_counts()

1    500
Name: Class, dtype: int64

In [196]:
class_0["Class"].value_counts()

0    500
Name: Class, dtype: int64

In [197]:
pima_oversamp= pd.concat([class_1_oversamp, class_0])

In [198]:
pima_oversamp.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
227,3,162,52,38,0,37.2,0.652,24,1
53,8,176,90,34,300,33.7,0.467,58,1


In [199]:
pima_oversamp["Class"].value_counts()

1    500
0    500
Name: Class, dtype: int64

In [200]:
pima_oversamp.shape

(1000, 9)

# 2. Resampling(over_sampling) using sklearn

In [211]:
from sklearn.utils import resample

In [212]:
class_1_oversamp= resample(class_1, replace= True, n_samples= len(class_0)) 

In [213]:
class_1_oversamp.shape

(500, 9)

In [214]:
class_0.shape

(500, 9)

In [215]:
pima_sklearn_over= pd.concat([class_1_oversamp, class_0])

In [216]:
pima_sklearn_over.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
143,10,108,66,0,0,32.4,0.272,42,1
114,7,160,54,32,175,30.5,0.588,39,1


In [217]:
pima_sklearn_over["Class"].value_counts()

1    500
0    500
Name: Class, dtype: int64

In [218]:
pima_sklearn_over.shape

(1000, 9)

# 3. over_sampling using Imblearn

Note: we can do over sampling using imblearn RandomOverSampler(its again does duplicating data)
      
      But if we use SMOTE(Synthetic Minority Over-sampling Technique) or ADASYN (Adaptive Synthetic Minority Over-sampling
      Technique) creates the Synthetic data (avoids duplicating data). Its the best way to avoid over fitting the model and 
      its generalises the model.

In [9]:
from imblearn.over_sampling import RandomOverSampler

In [14]:
ros= RandomOverSampler(sampling_strategy="auto", random_state=42)

In [15]:
X_resampled, y_resampled= ros.fit_resample(X,y)

In [21]:
pima_imblearn_over= pd.concat([X_resampled, y_resampled], axis=1)

In [22]:
pima_imblearn_over.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [23]:
pima_imblearn_over["Class"].value_counts()

1    500
0    500
Name: Class, dtype: int64

In [24]:
pima_imblearn_over.shape

(1000, 9)

# 4. over_sampling using imblearn- SMOTE

Note: it creates the synthetic data for over sampling unlike the above techniques. Best practice for a generalized model.

In [5]:
from imblearn.over_sampling import SMOTE

In [15]:
smote= SMOTE(sampling_strategy="auto", random_state=42)

In [16]:
X_resampled, y_resampled= smote.fit_resample(X, y)

In [17]:
pima_smote_over= pd.concat([X_resampled, y_resampled], axis=1)

In [18]:
pima_smote_over.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [19]:
pima_smote_over.shape

(1000, 9)

In [20]:
pima_smote_over["Class"].value_counts()

1    500
0    500
Name: Class, dtype: int64

# 5. over_sampling using imblearn- SVMSMOTE

In [31]:
from imblearn.over_sampling import SVMSMOTE

In [32]:
svm_smote= SVMSMOTE(sampling_strategy="auto", random_state=42)

In [33]:
X_resampled, y_resample= svm_smote.fit_resample(X,y)

In [34]:
pima_svmsmote_over= pd.concat([X_resampled, y_resampled], axis=1)

In [35]:
pima_svmsmote_over.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


In [36]:
pima_svmsmote_over.shape

(1000, 9)

In [37]:
pima_svmsmote_over["Class"].value_counts()

1    500
0    500
Name: Class, dtype: int64

Note: We can use different different parameters while applying SMOTE: i have given below some paramters to consider:

In [47]:
smote= SMOTE(sampling_strategy="auto",
             random_state=42,
             k_neighbors= 10,  # this value typically between 3 to 10.(its considers the majority and minority neighbours)
             #m_neighbors=10, # its also typical value between 3 to 10 (its considers only minority class neightbours)
             #kind= "regular", # borderline1, borderline2, svm
             n_jobs= -1   # it tell about how many cpu can be used -1 indicates all cpus
             )



In [43]:
import imblearn
print(imblearn.__version__)

0.11.0


In [46]:
svm_smote= SVMSMOTE(sampling_strategy="auto",
             random_state=42,
             k_neighbors= 10,  # this value typically between 3 to 10.(its considers the majority and minority neighbours)
             m_neighbors=10, # its also typical value between 3 to 10 (its considers only minority class neightbours)
             #kind= "regular", # borderline1, borderline2, svm
             n_jobs= -1   # it tell about how many cpu can be used -1 indicates all cpus
             )
